In [1]:
pwd

'/content'

In [2]:
from google.colab import drive
drive.mount('./drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./drive


Bism  
# Imports

In [0]:
import pandas as pd
import numpy as np
import os
import IPython.display as ipd
import librosa
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('ggplot')
from tqdm import tqdm_notebook


#SKLEARN
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as sklm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE

#KERAS
from keras import models
from keras.models import Model, load_model
from keras import callbacks
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, AveragePooling2D
from keras.optimizers import SGD, Adam

# Implementation 3  (PHONE LEVEL)
## Using Cambridge as Training set
## L2 as val+test set

In [0]:
phones = np.load('./drive/My Drive/Thesis Work/phoneList.npy',allow_pickle=True)
phones = phones[~np.isin(phones,['sil','sp','spn','AX','ZH'])]

In [22]:
len(phones)

39

### Training

#### X_TRAIN

In [0]:
X_train = np.load("./drive/My Drive/Thesis Work/Cambridge_bigger Phones Train/"+phones[0]+"_X.npy",allow_pickle=True)
for phone in phones[1:]:
  X_train = np.vstack([X_train,np.load("./drive/My Drive/Thesis Work/Cambridge_bigger Phones Train/"+phone+"_X.npy",allow_pickle=True)])

In [17]:
X_train.shape

(140121, 13, 50, 3)

_______

#### Y_TRAIN

In [0]:
y_train = [phones[0] for i in range(np.load("./drive/My Drive/Thesis Work/Cambridge_bigger Phones Train/"+phones[0]+"_X.npy",allow_pickle=True).shape[0])]
for phone in phones[1:]:
  y_train = np.hstack([y_train,[phone for i in range(np.load("./drive/My Drive/Thesis Work/Cambridge_bigger Phones Train/"+phone+"_X.npy",allow_pickle=True).shape[0])]])

In [25]:
y_train.shape

(140121,)

In [27]:
encoder = LabelBinarizer()
y_train = encoder.fit_transform(y_train)
y_train.shape

(140121, 39)

#### MODEL

In [30]:
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3),padding='same', activation='relu', input_shape=(13, 50, 3)))
model.add(Conv2D(32, (3, 3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(39, activation='softmax'))


callbacklist=[callbacks.History(),callbacks.ModelCheckpoint('./jawadmodel.h5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=5)]
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=2048, epochs=500, callbacks=callbacklist)

W0620 04:47:16.592694 139847307270016 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0620 04:47:16.661800 139847307270016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/500
140121/140121 [==============================] - 23s 165us/step - loss: 3.7567 - acc: 0.1184
Epoch 2/500
140121/140121 [==============================] - 13s 91us/step - loss: 2.4623 - acc: 0.3379
Epoch 3/500
140121/140121 [==============================] - 13s 92us/step - loss: 1.4311 - acc: 0.5737
Epoch 4/500
140121/140121 [==============================] - 13s 93us/step - loss: 1.1172 - acc: 0.6570
Epoch 5/500
140121/140121 [==============================] - 13s 94us/step - loss: 0.9985 - acc: 0.6924

Epoch 00005: loss improved from inf to 0.99849, saving model to ./jawadmodel.h5
Epoch 6/500
140121/140121 [==============================] - 13s 95us/step - loss: 0.9201 - acc: 0.7162
Epoch 7/500
140121/140121 [==============================] - 13s 94us/step - loss: 0.8681 - acc: 0.7311
Epoch 8/500
140121/140121 [==============================] - 13s 94us/step - loss: 0.8257 - acc: 0.7437
Epoch 9/500
140121/140121 [==============================] - 13s 95us/step - loss: 0.7

KeyboardInterrupt: ignored

In [31]:
callbacklist=[callbacks.History(),callbacks.ModelCheckpoint('./jawadmodel.h5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)]
model.fit(X_train, y_train, batch_size=2048, epochs=2, callbacks=callbacklist)

Epoch 1/2
140121/140121 [==============================] - 13s 95us/step - loss: 0.2246 - acc: 0.9255

Epoch 00001: loss improved from inf to 0.22458, saving model to ./jawadmodel.h5
Epoch 2/2
140121/140121 [==============================] - 13s 94us/step - loss: 0.2229 - acc: 0.9258

Epoch 00002: loss improved from 0.22458 to 0.22286, saving model to ./jawadmodel.h5


AH

In [0]:
!cp ./jawadmodel.h5 "./drive/My Drive/Thesis Work/bigger.h5"

### TESTING

#### Representation Layer, L2 norm, Prediction Checks

In [6]:
model = load_model("./drive/My Drive/Thesis Work/bigger.h5")

W0621 05:33:23.380001 139962515666816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0621 05:33:23.431215 139962515666816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0621 05:33:23.470636 139962515666816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0621 05:33:23.497031 139962515666816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0621 05:33:23.559725 139962515666816 deprecation_wrapper.py:119] From /u

In [33]:
y_train.shape

(140121, 39)

In [34]:
encoder.inverse_transform(model.predict(X_train[:4])) #predicting correctly

array(['AA', 'AA', 'AA', 'AA'], dtype='<U2')

In [35]:
encoder.inverse_transform(y_train[:4])

array(['AA', 'AA', 'AA', 'AA'], dtype='<U2')

In [0]:
#np.all(encoder.inverse_transform(y_train) == train_phones) #decoder also working fine

In [16]:
model.layers[-2] #representation

In [0]:
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(index=-2).output)

In [41]:
np.linalg.norm(intermediate_layer_model.predict(X_train[1:2]) - intermediate_layer_model.predict(X_train[1:2])) #l2 norm

0.0

In [43]:
intermediate_layer_model.predict(X_train[0:1]).shape

(1, 50)

#### LOADING ARRAYS, VALIDATION AND TESTING

In [0]:
train_phones = [phones[0] for i in range(np.load("./drive/My Drive/Thesis Work/Cambridge_bigger Phones Train/"+phones[0]+"_X.npy",allow_pickle=True).shape[0])]
for phone in phones[1:]:
  train_phones = np.hstack([train_phones,[phone for i in range(np.load("./drive/My Drive/Thesis Work/Cambridge_bigger Phones Train/"+phone+"_X.npy",allow_pickle=True).shape[0])]])

In [9]:
train_phones.shape

(140121,)

In [54]:
train_representations = intermediate_layer_model.predict(X_train)
train_representations.shape

(140121, 50)

In [56]:
train_representations[train_phones=='D'].shape

(5197, 50)

In [0]:
for phone in phones:
  np.save("./drive/My Drive/Thesis Work/PhoneLevel_RL_MFCC_TrainBigCamb_TestL2/meanVec_"+phone+".npy",np.mean(train_representations[train_phones==phone],axis=0))

In [0]:
test_typeOfError = np.load('./drive/My Drive/Thesis Work/PhoneLevel_RL_MFCC_TrainCamb_TestL2/test_typeOfError.npy',allow_pickle=True)
#only substitution errors
x_test = np.load('./drive/My Drive/Thesis Work/PhoneLevel_RL_MFCC_TrainCamb_TestL2/x_test.npy',allow_pickle=True)#[np.isin(test_typeOfError,[' s','S','s','c'])]
test_labels = np.load('./drive/My Drive/Thesis Work/PhoneLevel_RL_MFCC_TrainCamb_TestL2/test_labels.npy',allow_pickle=True)#[np.isin(test_typeOfError,[' s','S','s','c'])]
test_phones = np.load('./drive/My Drive/Thesis Work/PhoneLevel_RL_MFCC_TrainCamb_TestL2/test_phones.npy',allow_pickle=True)#[np.isin(test_typeOfError,[' s','S','s','c'])]

In [0]:
shuffledIndexes = np.random.permutation(range(x_test.shape[0]))
x_test = x_test[shuffledIndexes]
test_labels = test_labels[shuffledIndexes]
test_phones = test_phones[shuffledIndexes]

In [17]:
test_phones.shape #l2

(34799,)

In [20]:
set([p for p in test_phones if p not in train_phones]) #Only AX not in train

{'AX', 'sil', 'sp', 'spn'}

In [18]:
test_representations = intermediate_layer_model.predict(x_test)
test_representations.shape

(34799, 50)

In [19]:
np.ceil(sum(test_phones == phones[0])*40/100)

181.0

In [43]:
#VALIDATION
thresholdList = []
for phone in phones:
  print("\n\n------->>>>>>Phone: ", phone)
  print()
  
  valSetLength = np.ceil(sum(test_phones==phone)*40/100)
  
  print("Total Samples validation: ", valSetLength)
  
  truelabels = test_labels[test_phones==phone][:int(valSetLength)]
  accs = []
  f1mis = []
  train_representation = np.load("./drive/My Drive/Thesis Work/PhoneLevel_RL_MFCC_TrainBigCamb_TestL2/meanVec_"+phone+".npy")
  
  print("\n-----------THRESHOLDING-----------\n")
  
  y_l2s = np.linalg.norm(train_representation - test_representations[test_phones==phone][:int(valSetLength)],axis=1)
  
  print("Total Thresholds: ", y_l2s.size)
  
  for threshold in sorted(y_l2s):
    predictedLabels = 1*(y_l2s <= threshold)
    accs.append(sklm.accuracy_score(truelabels,predictedLabels))
    f1mis.append(sklm.precision_recall_fscore_support(truelabels,predictedLabels)[2][0])
  
  thresholdList.append(sorted(y_l2s)[np.argmax(f1mis)])
  
  
  print("Max Achievable Accuracy (Overall): ", max(accs))
  print("Max Achievable Accuracy (Overall) on Threshold: ", sorted(y_l2s)[np.argmax(accs)])

  print("Max Achievable F-1 score Threshold: ", sorted(y_l2s)[np.argmax(f1mis)])
  print("Max Achievable Accuracy On This Threshold: ", accs[np.argmax(f1mis)])
  print("Max Achievable F-1 score for Mispronunciations: ", max(f1mis))
  print("============================================")



------->>>>>>Phone:  AA

Total Samples validation:  181.0

-----------THRESHOLDING-----------

Total Thresholds:  181
Max Achievable Accuracy (Overall):  0.8176795580110497
Max Achievable Accuracy (Overall) on Threshold:  68.763695
Max Achievable F-1 score Threshold:  42.41554
Max Achievable Accuracy On This Threshold:  0.7569060773480663
Max Achievable F-1 score for Mispronunciations:  0.4358974358974359


------->>>>>>Phone:  R



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Total Samples validation:  580.0

-----------THRESHOLDING-----------

Total Thresholds:  580


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Max Achievable Accuracy (Overall):  0.8586206896551725
Max Achievable Accuracy (Overall) on Threshold:  84.737335
Max Achievable F-1 score Threshold:  41.79323
Max Achievable Accuracy On This Threshold:  0.5862068965517241
Max Achievable F-1 score for Mispronunciations:  0.29411764705882354


------->>>>>>Phone:  EY

Total Samples validation:  245.0

-----------THRESHOLDING-----------

Total Thresholds:  245


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Max Achievable Accuracy (Overall):  0.9166666666666666
Max Achievable Accuracy (Overall) on Threshold:  79.99531
Max Achievable F-1 score Threshold:  65.1944
Max Achievable Accuracy On This Threshold:  0.90625
Max Achievable F-1 score for Mispronunciations:  0.39999999999999997


------->>>>>>Phone:  UH

Total Samples validation:  64.0

-----------THRESHOLDING-----------

Total Thresholds:  64
Max Achievable Accuracy (Overall):  0.890625
Max Achievable Accuracy (Overall) on Threshold:  61.798386
Max Achievable F-1 score Threshold:  45.23618
Max Achievable Accuracy On This Threshold:  0.796875
Max Achievable F-1 score for Mispronunciations:  0.23529411764705882


------->>>>>>Phone:  OY

Total Samples validation:  24.0

-----------THRESHOLDING-----------

Total Thresholds:  24
Max Achievable Accuracy (Overall):  0.9583333333333334
Max Achievable Accuracy (Overall) on Threshold:  93.22568
Max Achievable F-1 score Threshold:  58.159
Max Achievable Accuracy On This Threshold:  0.7083333333

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Total Samples validation:  389.0

-----------THRESHOLDING-----------

Total Thresholds:  389
Max Achievable Accuracy (Overall):  0.519280205655527
Max Achievable Accuracy (Overall) on Threshold:  28.507042
Max Achievable F-1 score Threshold:  28.21249
Max Achievable Accuracy On This Threshold:  0.5167095115681234
Max Achievable F-1 score for Mispronunciations:  0.6630824372759857


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [50]:
#Testing
f1ScoresList = []
for t,phone in enumerate(phones):
  if phone != 'OY':
    print("\n\n------->>>>>>Phone: ", phone)
    print()

    valSetLength = np.ceil(sum(test_phones==phone)*40/100)

    print("Total Samples Test: ", sum(test_phones==phone)-valSetLength)

    truelabels = test_labels[test_phones==phone][int(valSetLength):]
    train_representation = np.load("./drive/My Drive/Thesis Work/PhoneLevel_RL_MFCC_TrainBigCamb_TestL2/meanVec_"+phone+".npy")

    print("\n-----------TESTING-----------\n")

    y_l2s = np.linalg.norm(train_representation - test_representations[test_phones==phone][int(valSetLength):],axis=1)

    predictedLabels = 1*(y_l2s <= thresholdList[t])

    print(classification_report(truelabels, predictedLabels, target_names=['Mispronounced','Correct']))
    print(confusion_matrix(truelabels, predictedLabels, labels=range(2)))

    print(sklm.roc_curve(truelabels,predictedLabels))
    print('FAR = ', sklm.roc_curve(truelabels,predictedLabels)[0][1]*100,'%')
    print('FRR = ', (1 - sklm.roc_curve(truelabels,predictedLabels)[1][1])*100,'%')
    print("AUC_SCORE = ", sklm.roc_auc_score(truelabels,predictedLabels)*100,'%')
    print("============================================")
    f1ScoresList.append(sklm.precision_recall_fscore_support(truelabels,predictedLabels)[2][0])



------->>>>>>Phone:  AA

Total Samples Test:  270.0

-----------TESTING-----------

               precision    recall  f1-score   support

Mispronounced       0.20      0.30      0.24        43
      Correct       0.85      0.77      0.81       227

     accuracy                           0.70       270
    macro avg       0.53      0.54      0.53       270
 weighted avg       0.75      0.70      0.72       270

[[ 13  30]
 [ 52 175]]
(array([0.        , 0.69767442, 1.        ]), array([0.        , 0.77092511, 1.        ]), array([2, 1, 0]))
FAR =  69.76744186046511 %
FRR =  22.90748898678414 %
AUC_SCORE =  53.66253457637538 %


------->>>>>>Phone:  R

Total Samples Test:  868.0

-----------TESTING-----------

               precision    recall  f1-score   support

Mispronounced       0.21      0.62      0.31       138
      Correct       0.89      0.56      0.68       730

     accuracy                           0.57       868
    macro avg       0.55      0.59      0.50       868


In [52]:
f1ScoresList

[0.24074074074074073,
 0.31444241316270566,
 0.30000000000000004,
 0.11851851851851852,
 0.038461538461538464,
 0.23547400611620797,
 0.14559386973180077,
 0.10256410256410256,
 0.2222222222222222,
 0.0,
 0.3318777292576419,
 0.20164609053497942,
 0.42887473460721864,
 0.15686274509803924,
 0.35751295336787564,
 0.027586206896551724,
 0.28571428571428575,
 0.14358974358974358,
 0.23280423280423282,
 0.4210526315789474,
 0.24000000000000002,
 0.3783783783783784,
 0.16326530612244897,
 0.047619047619047616,
 0.08000000000000002,
 0.14096916299559473,
 0.5185185185185185,
 0.6415500538213132,
 0.1286549707602339,
 0.0,
 0.3010752688172043,
 0.27906976744186046,
 0.52,
 0.0,
 0.13333333333333333,
 0.19354838709677416,
 0.6674445740956826]

In [53]:
print("Mean F1 Score: ", round(np.mean(f1ScoresList),2)*100,"%")

Mean F1 Score:  24.0 %
